<a href="https://colab.research.google.com/github/trulicups/mindmap/blob/main/courses/machine_learning/introduction_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# definitions
- **untrained model** - a machine learning algorithm that has not been trained
- **trained model** - a machine leaning model that has been trained to perform some type of predictions
- **unsupervised learning** - leaning with an unlabeled dataset
  - **unlabeled training dataset** - dataset where the data does not have labels
- **supervised learning** - learning with a labeled dataset
  - **labeled training dataset** - dataset where the data is labeled

- **inference** - deriving predictions from a trained model
  - **classification** - predicting the category of a datum from a finite set of categories of interest

In [ ]:
!nvidia-smi

In [ ]:
import torch
if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"
device = torch.device(dev)
dev

In [ ]:
!pip install pytorch-lightning # pip - package installer for python

In [ ]:
!ls

# computer vision library
`torch` is the basic Pytorch deep leaning library and `torchvision` is the library containing models and other components for computer vision.

In [ ]:
import torch
from torchvision import models

In [ ]:
dir(models) # list all the objects in the `models` module

## load the pre-trained model

In [ ]:
model = models.alexnet(weights='AlexNet_Weights.DEFAULT')
# have model run on the GPU
model.cuda()

In [ ]:
next(model.parameters()).device

In [ ]:
pretrained_model = models.alexnet(pretrained=True)
historic_model = models.alexnet(pretrained=True)

## inference

In [ ]:
!curl http://zacharski.org/files/courses/dmpics/poodle.jpg -o poodle.jpg

In [ ]:
from PIL import Image
img = Image.open("poodle.jpg")

In [ ]:
img

## transform the image to match the model's input parameters

In [ ]:
from torchvision import transforms

# the image is originaly 4032x3024

transform = transforms.Compose([
    transforms.Resize(256), # scales the image to 341x256
    transforms.CenterCrop(224), # scales the image to 224x224
    transforms.ToTensor(), # transforms the image into a 224x224x3 array
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]) # normalize the image with the mean and standard deviation of the training dataset
])

In [ ]:
img_transformed = transform(img)

# adds img_transformed to an array for batch processing
batch_transform = torch.unsqueeze(img_transformed, 0)

# put the tensor on the GPU
batch_transform = batch_transform.to(device)
batch_transform.device

# model inference 
in Pytorch, models can be in two modes and we can toggle between them.
- `model.eval()` puts the model in inference mode so it can make predictions.
- `model.train()` puts the model in training mode.

let's get the model ready for inference

In [ ]:
model.eval()

In [ ]:
# model inference
output = model(batch_transform)

print(output.shape)

In [ ]:
# raw predictions
output

In [ ]:
# normalized predictions
probability = torch.nn.functional.softmax(output, dim=1)
probability

In [ ]:
!curl http://zacharski.org/files/courses/dmpics/imagenet_classes.txt -o model_classes.txt

In [ ]:
# the `with` statement does error handling within its scope
with open('model_classes.txt') as f:

  labels = [line.strip() for line in f.readlines()]
print(labels[:5]) # print the first 5 labels

In [ ]:
z, index = torch.max(output, 1)
print(index)

In [ ]:
print(labels[index[0]], probability[0][index[0]].item())

In [ ]:
_, indices = torch.sort(output, descending=True)
[(labels[idx], probability[0][idx].item()) for idx in indices[0][:5]]

In [ ]:
import requests

def predict(url):
  # download the image srom the internet
  r = requests.get(url)

  # write the image in the url to a temporary jpg
  with open('tmp.jpg', 'wb') as f:
    f.write(r.content)

  # load the temporary image into a variable
  img = Image.open('tmp.jpg')
  img.show()

  # transform and batch the image to match the model's input parameter
  img_transformed = transform(img)
  batch_transformed = torch.unsqueeze(img_transformed, 0)

  # add a GPU device
  batch_transformed = batch_transformed.to(device)

  # predict the label of the image
  output = model(batch_transformed)

  # sort the output tensor in descending order and normalize the values
  _, indices = torch.sort(output, descending=True)
  probability = torch.nn.functional.softmax(output, dim=1)[0]

  return([(labels[idx], probability[idx].item()) for idx in indices[0][:5]])

In [ ]:
line = ''

for i in range(len(labels)):
  line += '%-35s' %labels[i]
  if (i + 1) % 4 == 0:
    print(line)
    line = ''

In [ ]:
predict('https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/Fender_Stratocaster.jpeg')

In [ ]:
predict('https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/greyOwl.jpeg')

In [ ]:
predict('https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/cello12.jpeg')

In [ ]:
predict('https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse2.mm.bing.net%2Fth%3Fid%3DOIP.XrEWU-B7559j1hf9Tzo1RgHaHK%26pid%3DApi&f=1&ipt=4c9b0aa3a32d41c79385bfca366b945d8c2cf589dcc348eff5f46781b6be554d&ipo=images')

In [ ]:
predict('https://external-content.duckduckgo.com/iu/?u=http%3A%2F%2Fdreamicus.com%2Fdata%2Ffig%2Ffig-01.jpg&f=1&nofb=1&ipt=e5eb537001a830933d12a99f620395c0c1b9c7ae5dd26b453d9cf064afc8df18&ipo=images')

In [ ]:
predict('https://external-content.duckduckgo.com/iu/?u=http%3A%2F%2F4.bp.blogspot.com%2F-cI_xGP3b3GU%2FUhHHkTUjoYI%2FAAAAAAAAGZo%2F7dxJQdoKIbs%2Fs1600%2FCrocodile-African-2013-01.jpg&f=1&nofb=1&ipt=4e18603b849f3e6d12b3a562da93f186f04d6893702701e007462aea67fe50f0&ipo=images')